TF 2.0 brings together the ease of eager execution and the power of TF 1.0. At the center of this merger is `tf.function`, which allows you to transform a subset of Python syntax into portable, high-performance TensorFlow graphs.

A cool new feature of `tf.function` is Autograph Autograph: Conversion of plain Python into TensorFlow graph code.,
which lets you write graph code using natural Python syntax. for a list of the Python features tha t you can use with AutoGraph, see AutoGraph Capabilities and Limitations. For more details about `tf.function`, see the RFC `TF2.0: Functions, not Sessions. For more details about AutoGraph, ses `tf.autograph`.

This tutorial will walk you through the basic features of `tf.function` and AutoGraph.



In [0]:
%tensorflow_version 2.x
import numpy as np
import tensorflow as tf

## The `tf.function` decorater
When you annotate a function with `tf.function`, you can still call it like any other function. But it will be compiled into a graph, which means you get the benefits of faster execution, running on GPU or TPY, or exporting to SaveModel.

In [0]:
@tf.function
def simple_nn_layer(x, y):
  return tf.nn.relu(tf.matmul(x, y))

x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

simple_nn_layer(x, y)

If we examine the result of the annotation, we can see that it's a special callable that handles all interations with the TensorFlow runtime.

In [0]:
simple_nn_layer

If your code uses multiple functions, you don't need to annotate them all - any functions called from an annotated function will also run in graph mode.

In [0]:
def linear_layer(x):
  return 2 * x + 1

@tf.function
def deep_net(x):
  return tf.nn.relu(linear_layer(x))

deep_net(tf.constant((1, 2, 3)))

Functions can be faster than eager code, for graphs with many small ops. But for graphs with a few expensive ops (like convolutions), you may not see much speedup.

In [0]:
import timeit

conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)
  
image = tf.zeros([1, 200, 200, 100])

# warm up
conv_layer(image); conv_fn(image)

print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print('Function conv:', timeit.timeit(lambda: conv_fn(image), number=10))
print("Note how there's not much difference in performance for convolutions")

In [0]:
lstm_cell = tf.keras.layers.LSTMCell(10)

@tf.function
def lstm_fn(input, state):
  return lstm_cell(input, state)

input = tf.ones([10, 10])
state = [tf.ones([10, 10])] * 2

# Warm up
# lstm_cell(input, state); lstm_fn(input, state)

print('eager lstm:', timeit.timeit(lambda: lstm_cell(input, state), number=10))
print('function lstm:', timeit.timeit(lambda: lstm_fn(input, state), number=10))

In [0]:
# After warm up

print('eager lstm:', timeit.timeit(lambda: lstm_cell(input, state), number=10))
print('function lstm:', timeit.timeit(lambda: lstm_fn(input, state), number=10))

## Use Python control flow
When using data-dependent control flow inside `tf.function`, you can use Python control flow statements and AutoGraph will convert them into appropriate TensorFlow ops. For example, `if` statements will be converted into `tf.cond()` if they depend on a `Tensor`.

In [0]:
# In the example below, `x` is a `Tensor` but the `if` statement works as expected
@tf.function
def square_if_positive(x):
  if x > 0:
    x = x * x
  else:
    x = 0
  return x

print(square_if_positive(tf.constant(2)))
print(square_if_positive(tf.constant(-2)))

Note: The previous uses simple conditional with scalar values. Batching is typically used in real-world code.

AutoGraph support common Python statements like `while`, `for`, `if`, `break`, `continue`, and `return`, with support for nesting. That means you can use `Tensor` expressions in the condition of `while` and `if` statements, or iterate over a `Tensor` in a `for` loop

In [0]:
@tf.function
def sum_even(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s

sum_even(tf.constant([10, 12, 15, 20]))

AutoGraph also provides a low-level API for advanced users. For example we can use it to have a look at the generated code.

In [0]:
print(tf.autograph.to_code(sum_even.python_function))

In [0]:
# Here's an example of more complicated control flow
@tf.function
def fizzbuzz(n):
  for i in tf.range(n):
    if i % 3 == 0:
      tf.print('Fizz')
    elif i % 5 == 0:
      tf.print('Buzz')
    else:
      tf.print(i)

fizzbuzz(tf.constant(15))

## Keras and AutoGraph
AutoGraph is available by default in non-dynamic Keras model. For more information, see `tf.keras`.

In [0]:
class CustomModel(tf.keras.models.Model):
  
  @tf.function
  def call(self, input_data):
    if tf.reduce_mean(input_data) > 0:
      return input_data
    else:
      return input_data // 2

model = CustomModel()

model(tf.constant([-2, -4]))

## Side effects
Just like in eager mode, you can use operations with side effects, like `tf.assign` or `tf.print` normally inside `tf.function`, and it will insert the necessary control dependencies to ensure they execute in order.

In [0]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
  v.assign(v + 1)
  if v % 2 == 0:
    v.assign(v + 1)

find_next_odd()
v

## Debugging
`tf.function` and AutoGraph work by generating code and tracing it into TensorFlow graphs. This mechanism does not yet support step-by-step debuggers like `pdb`. However, you can call `tf.config.run_functions_eagerly(True)` to temporarily enable eager execution inside the `tf.function` and use your favorate debugger:

In [0]:
@tf.function
def f(x):
  if x > 0:
    import pdb
     pdb.set_trace()
     x = x + 1
  return x

tf.config.experimental_run_functions_eagerly(True)

# You can now set breakpoints and run the code in a debugger.
f(tf.constant(1))

tf.config.experimental_run_functions_eagerly(False)

## Advanced example: An in-graph training loop

The previous section showed that AutoGraph can be used inside Keras layers and models. Keras models can also be used in AutoGraph code.

This example shows how to train a simple Keras model on MNIST with the entire training process - loading batches, calculating gradients, updating parameters, calculating validation accuracy, and repeating untill convergence - is performed in-graph

In [0]:
# Download data
def prepare_mnist_features_and_labels(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  y = tf.cast(y, tf.int64)
  return x, y

def mnist_dataset():
  (x, y), _ = tf.keras.datasets.mnist.load_data()
  ds = tf.data.Dataset.from_tensor_slices((x, y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.take(20000).shuffle(20000).batch(100)
  return ds

train_dataset = mnist_dataset()

In [0]:
# Define the model
model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))

model.build()
optimizer = tf.keras.optimizers.Adam()

In [0]:
print(type(1))
print((1))
print(type(1,))
print((1,))

### Define the training loop

In [0]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


def train_one_step(model, optimizer, x, y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(y, logits)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  compute_accuracy(y, logits)
  return loss


@tf.function
def train(model, optimizer):
  train_ds = mnist_dataset()
  step = 0
  loss = 0.0
  accuracy = 0.0
  for x, y in train_ds:
    step += 1
    loss = train_one_step(model, optimizer, x, y)
    if step % 10 == 0:
      tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
  return step, loss, accuracy

step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())

## Batching

In real applications batching is essential for performance. The best code to convert to AutoGraph is code where the control flow is decided at the batch level. If making decisions at the individual example level, try to use batch APIs to maintain performace.

In [0]:
# For example, if you have the follwing code in Python
def square_if_positive(x):
  return [i ** 2 if i > 0 else i for i in x]

square_if_positive(range(-5, 5))

In [0]:
# You may be tempted to write it in TensorFlow as such (and this would work!):
@tf.function
def square_if_positive_naive(x):
  result = tf.TensorArray(tf.int32, size=x.shape[0])
  for i in tf.range(x.shape[0]):
    if x[i] > 0:
      result = result.write(i, x[i] ** 2)
    else:
      result = result.write(i, x[i])
  return result.stack()


square_if_positive_naive(tf.range(-5, 5))

In [0]:
# But in this case, it turns out you can write the following
def square_if_positive_vectorized(x):
  return tf.where(x > 0, x ** 2, x)

square_if_positive_vectorized(tf.range(-5, 5))